In [1]:
# import matplotlib
# matplotlib.use('Agg')
from docx import Document
from docx.shared import Inches
document = Document()
%matplotlib inline
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# import matplotlib.gridspec as gridspec
from matplotlib import cm
from scipy.spatial import distance
import numpy as np
import pandas as pd
import pickle
import os
import gc



In [2]:
def spk_times():
    cell_type = ['et', 'apimi', 'mi', 'gr'] 
    binL = 25
    nrows = 1
    ncols = 3
    os.chdir(add)
    for cidx, ctype in enumerate(cell_type):
        if ctype in ['apimi', 'mi']:
            data = [[[] for _ in range(param["mi_dup"])] for _ in range(6)]
            avg_data = np.zeros((int(param['num_mi']/param["mi_dup"]), nclasses, param["mi_dup"]))
            cnt = [[[] for _ in range(param["mi_dup"])] for _ in range(6)]
        elif ctype in ['gr']:
            data = [[] for _ in range(nclasses)]
            cnt = [[] for _ in range(nclasses)]
            # f_cnt = [[[] for _ in range(nclasses)] for _ in range(6)]
            list_cnt = [[[] for _ in range(nclasses)] for _ in range(6)]
        '''
        elif ctype in ['etach']:
            v_etach_train = [[] for _ in range(nclasses)]
            v_etach_val = [[] for _ in range(nclasses)]
            v_etach_test = [[] for _ in range(nclasses)]
        '''
        if ctype in ['et']:
            cnt_train = [[] for _ in range(nclasses)]
            cnt_val = [[] for _ in range(nclasses)]
            cnt_test = [[] for _ in range(nclasses)]
            
        for gidx in range(len(group_type)):
            for st in range(len(stat)):
                if st == 0:
                    y = y_train[gidx]
                elif st == 1:
                    y = y_val[gidx]
                elif st == 2:
                    y = y_test[gidx]  
                for s_idx in range(len(y)):
                    if st == 0:
                        if ctype in ['apimi', 'mi', 'gr']:
                            t_ctype = pd.read_csv('spk_time_' + ctype + stat[st] + str(gidx) + 
                                                  str(s_idx) + '.csv', header=None)
                            t_ctype = np.asarray(t_ctype.values.flatten())
                            if ctype in ['apimi', 'mi']:
                                for i in range(param['mi_dup']):
                                    num_mi = param['num_mi']/param["mi_dup"]
                                    data[y[s_idx]-1][i].append(t_ctype[int(i*num_mi): int((i*num_mi) + num_mi)])
                                    avg_data[:, y[s_idx]-1, i] = (avg_data[:, y[s_idx]-1, i] + 
                                                                  t_ctype[int(i*num_mi): int((i*num_mi) + num_mi)])
                            elif ctype in ['gr']:
                                data[y[s_idx]-1].append(t_ctype)

                            t_ctype = pd.read_csv('count_' + ctype + stat[st] + str(gidx) + 
                                                  str(s_idx) + '.csv', header=None)
                            t_ctype = np.asarray(t_ctype.values.flatten())
                            if ctype in ['apimi', 'mi']:
                                for i in range(param['mi_dup']):
                                    num_mi = param['num_mi']/param["mi_dup"]
                                    cnt[y[s_idx]-1][i].append(np.sum(
                                        t_ctype[int(i*num_mi): int((i*num_mi) + num_mi)]))
                            elif ctype in ['gr']:
                                cnt[y[s_idx]-1].append(np.sum(t_ctype))
                                for gr_idx in range(nclasses):
                                    '''
                                    f_cnt[y[s_idx]-1][gr_idx].append(
                                        np.sum(t_ctype[param["gr_orth_idx"][gr_idx+1]]))
                                    '''
                                    list_cnt[y[s_idx]-1][gr_idx].append(
                                        np.sum(t_ctype[param["idx_list_all"][gr_idx]]))
                                
                        if ctype in ['et']:
                            t_ctype = pd.read_csv('count_' + ctype + stat[st] + str(gidx) + 
                                                  str(s_idx) + '.csv', header=None)
                            t_ctype = np.asarray(t_ctype.values.flatten())
                            if st == 0:
                                cnt_train[y[s_idx]-1].append(t_ctype)    
                            elif st == 1:
                                cnt_val[y[s_idx]-1].append(t_ctype)
                            elif st == 2:
                                cnt_test[y[s_idx]-1].append(t_ctype)
                                        
                    else:
                        if gidx == 5:
                            if ctype in ['apimi', 'mi', 'gr']:
                                t_ctype = pd.read_csv('spk_time_' + ctype + stat[st] + 
                                                      AL[st-1] + str(gidx) + 
                                                      str(s_idx) + '.csv', header=None)
                                t_ctype = np.asarray(t_ctype.values.flatten())
                                if ctype in ['apimi', 'mi']:
                                    for i in range(param['mi_dup']):
                                        num_mi = param['num_mi']/param["mi_dup"]
                                        data[y[s_idx]-1][i].append(
                                            t_ctype[int(i*num_mi): int((i*num_mi) + num_mi)])
                                        avg_data[:, y[s_idx]-1, i] = (avg_data[:, y[s_idx]-1, i] + 
                                                                  t_ctype[int(i*num_mi): int((i*num_mi) 
                                                                                             + num_mi)])
                                else:
                                    data[y[s_idx]-1].append(t_ctype)

                                t_ctype = pd.read_csv('count_' + ctype + stat[st] + AL[st-1] + str(gidx) + 
                                                      str(s_idx) + '.csv', header=None)
                                t_ctype = np.asarray(t_ctype.values.flatten())
                                if ctype in ['apimi', 'mi']:
                                    for i in range(param['mi_dup']):
                                        num_mi = param['num_mi']/param["mi_dup"]
                                        cnt[y[s_idx]-1][i].append(np.sum(
                                            t_ctype[int(i*num_mi): int((i*num_mi) + num_mi)]))
                                else:
                                    cnt[y[s_idx]-1].append(np.sum(t_ctype))
                                    for gr_idx in range(nclasses):
                                        '''
                                        f_cnt[y[s_idx]-1][gr_idx].append(np.sum(
                                            t_ctype[param["gr_orth_idx"][gr_idx+1]]))
                                        '''
                                        list_cnt[y[s_idx]-1][gr_idx].append(np.sum(
                                            t_ctype[param["idx_list_all"][gr_idx]]))
                                
                            if ctype in ['et']:
                                t_ctype = pd.read_csv('count_' + ctype + stat[st] +  
                                                      AL[st-1] + str(gidx) + 
                                                      str(s_idx) + '.csv', header=None)
                                t_ctype = np.asarray(t_ctype.values.flatten())
                                if st == 0:
                                    cnt_train[y[s_idx]-1].append(t_ctype)    
                                elif st == 1:
                                    cnt_val[y[s_idx]-1].append(t_ctype)
                                elif st == 2:
                                    cnt_test[y[s_idx]-1].append(t_ctype)
                            
        if ctype in ['et']:
            for st in range(len(stat)):
                if st == 0:
                    cnt = cnt_train
                elif st == 1:
                    cnt = cnt_val
                elif st == 2:
                    cnt = cnt_test
                color_list = ["g", "r", "b", "k", "y", "c"]
                id_0 = np.flipud(np.argsort(t_ctype[:]))
                fig, axes = plt.subplots(1, 3, figsize=(18,6)) # no of classes
                for i in range(nclasses):
                    for eg in range(np.size(cnt[i], 0)):
                        axes[0].plot(np.transpose(np.asarray(cnt[i])[eg, :][id_0]), color_list[i])
                axes[0].set_title("0. et/ntce2" + str(st))
                axes[0].set_xlabel("sensor indices")
                axes[0].set_ylabel("sensor responses")

                for i in range(nclasses):
                    for eg in range(np.size(cnt[i], 0)):
                        axes[1].plot(np.sort(np.asarray(cnt[i])[eg, :][::-1]), color_list[i])

                axes[1].set_title("1. et/ntce2 sorted" + str(st))

                ref_data = cnt_train[0][0][:]
                no_samples = 0
                for i in range(nclasses):
                    for eg in range(np.size(cnt[i], 0)):
                        axes[2].plot(no_samples, distance.euclidean(ref_data, cnt[i][eg][:]),marker='.', 
                                       linestyle='None', markersize=20, color = color_list[i])
                        no_samples =  no_samples + 1
                axes[2].set_title("2. euc_dist" + str(st))
                axes[2].set_xlabel("sample indices")
                axes[2].set_ylabel("dist")

                os.chdir(data_add)
                fig.savefig(ctype + 'et_resp' + str(st) + '.png')
                plt.close("all")

                document.add_heading('Unsupervised concentration invariance', level=1)
                p = document.add_paragraph()
                r = p.add_run()

                r.add_picture(ctype + 'et_resp' + str(st) + '.png', width=Inches(6.0), height=Inches(2.))
                r.add_text('Figure 0 shows the ET cell responses after application of unsupervised concentration invariance(NTCE2).\n')
                r.add_text('All Train, val, test set combined. X axis: sensor indices, Y axis: sensor responses.\n')
                r.add_text('Fig. 1: same as Fig. 0 but with the sensor responses sorted  - X axis indices are insignificant here.\n')
                r.add_text('Euclidean distances of ET cell responses w.r.t a reference sample - train set sample 0.\n')
                r.add_text('Note: NTCE2 computation is graded.')
                os.chdir(add)
                            
                                    
        if ctype in ['apimi', 'mi']:
            fig, axes = plt.subplots(1, max(param['mi_dup'], 2), figsize=(18,6))
            for j in range(param['mi_dup']):  
                color_list = ["g", "r", "b", "k", "y", "c"]
                for i in range(nclasses):
                    data[i][j] = np.asarray(data[i][j]).flatten()
                    bins = np.linspace(-5, 25, 75)
                    axes[j].hist(data[i][j], bins, color = color_list[i])
                    axes[j].text(16, 350, 'vth:' + format(param[ctype +"_v_th"][int(j*num_mi)], '.2f'))
                    if ctype == 'apimi':
                        axes[j].set_ylim([0, 450])
                    elif ctype == 'mi':
                        axes[j].set_ylim([0, 1800])
                axes[j].set_title(ctype + "stimes")
                axes[j].set_ylabel("#")
                axes[j].set_xlabel("spike times")
                
            os.chdir(data_add)
            fig.savefig(ctype + 'stimes' + '.png')
            plt.close("all")
        
            p = document.add_paragraph()
            r = p.add_run()
            r.add_picture(ctype + 'stimes' + '.png', width=Inches(6.0), height=Inches(2.))
            r.add_text(str(ctype) + '_spiketimes.\n')
            r.add_text('Spike timing distribution of all the input data (6 gas classes).\n')
            r.add_text('Fig. columns correspond to multiple spiking thresholds of mitral cells.\n')
            os.chdir(add)
            
            fig, axes = plt.subplots(1, max(param['mi_dup'], 2), figsize=(18,6))
            for j in range(param['mi_dup']):  
                color_list = ["g", "r", "b", "k", "y", "c"]
                for n in range(nclasses):
                    avg_data[:, n, j] = avg_data[:, n, j]/len(data[n][0])/num_mi
                img = axes[j].imshow(avg_data[:, :, j], interpolation='none')
            fig.colorbar(img)

            os.chdir(data_add)
            plt.savefig(ctype + str(j) + 'stimes_map' + '.png')
            plt.close("all")

            p = document.add_paragraph()
            r = p.add_run()
            r.add_text(str(ctype) + str(j) + '_avgspiketimes_map.\n')
            r.add_picture(ctype + str(j) + 'stimes_map' + '.png', width=Inches(6.0), height=Inches(2.))
            r.add_text('Average spike timings of cells w.r.t diff gas classes.\n')
            r.add_text('Fig. columns correspond to multiple spiking thresholds of mitral cells.\n')
            r.add_text('Y axis: cell indices with a particular threshold, X axis: Gas classes.')
            os.chdir(add)
               
        if ctype in ['gr']:
            fig = plt.figure()     
            color_list = ["g", "r", "b", "k", "y", "c"]
            for i in range(nclasses):
                data[i] = np.asarray(data[i]).flatten()
                bins = np.linspace(-5, 25, 75)
                plt.hist(data[i], bins, color = color_list[i])
            plt.title(ctype + "stimes")
            plt.title(ctype + "spike time distribution")
            plt.ylabel("#")
            plt.xlabel("spike times")
            
            os.chdir(data_add)
            fig.savefig(ctype + 'stimes' + '.png')
            plt.close("all")
        
            p = document.add_paragraph()
            r = p.add_run()
            r.add_text(str(ctype) + '_spiketimes ')
            r.add_picture(ctype + 'stimes' + '.png', width=Inches(6.0), height=Inches(2.))
            r.add_text('Granule cell spike timing distributions. ')
            os.chdir(add)
            
        if ctype in ['apimi', 'mi']:
            fig, axes = plt.subplots(1, max(param['mi_dup'], 2), figsize=(18,6))
            for j in range(param['mi_dup']): 
                color_list = ["g", "r", "b", "k", "y", "c"]
                for i in range(nclasses):
                    cnt[i][j] = np.asarray(cnt[i][j]).flatten()
                    axes[j].scatter(i* np.ones(len(cnt[i][j])) + 1, 
                                    np.asarray(cnt[i][j]), color = color_list[i])
                    axes[j].set_ylim([0, num_mi])
                    axes[j].text(3, 15, 'vth:' + format(param[ctype +"_v_th"][int(j*num_mi)], '.2f'))
                
                axes[j].set_title(ctype + "_cnt")
                axes[j].set_ylabel("#")
                axes[j].set_xlabel("spike count")
                
            os.chdir(data_add)
            fig.savefig(ctype + 'cnt' + '.png')
            plt.close("all")
            
            p = document.add_paragraph()
            r = p.add_run()
            r.add_picture(ctype + 'cnt' + '.png', width=Inches(6.0), height=Inches(2.))
            r.add_text(str(ctype) + '_spikecnt.\n')
            r.add_text('Scatter plots of spike count of all the input data (6 gas classes).\n')
            r.add_text('Fig. columns correspond to multiple spiking thresholds of mitral cells.\n')
            os.chdir(add)
                
        elif ctype in ['gr']:
            fig = plt.figure()     
            color_list = ["g", "r", "b", "k", "y", "c"]
            for i in range(nclasses):
                cnt[i] = np.asarray(cnt[i]).flatten()
                plt.scatter(i* np.ones(len(cnt[i])) + 1, np.asarray(cnt[i]), color = color_list[i])
            plt.title("Gr spike count")
            plt.xlabel(" Gas classes")
            
            os.chdir(data_add)
            fig.savefig(ctype + 'cnt' + '.png')
            plt.close("all")
        
            p = document.add_paragraph()
            r = p.add_run()
            r.add_text(str(ctype) + '_cnt')
            r.add_picture(ctype + 'cnt' + '.png', width=Inches(6.0), height=Inches(2.))
            r.add_text('Total Granule cell spike count for all the samples - this count includes Gr from all assigned Gr groups to classes.')
            os.chdir(add)
            
        if ctype in ['gr']:
            '''
            fig, axes = plt.subplots(1, nclasses, figsize=(18,6)) # no of classes
            for i in range(nclasses):
                for j in range(nclasses):
                    f_cnt[i][j] = np.asarray(f_cnt[i][j]).flatten()
                    axes[i].scatter(j* np.ones(len(f_cnt[i][j])) + 1, 
                                    f_cnt[i][j], color = color_list[i])
                    axes[j].set_ylim([0, len(param["gr_orth_idx"][1])])
                    axes[j].set_xlabel(" Class indices.")
            plt.title(ctype + "fcnt")
            
            os.chdir(data_add)
            fig.savefig(ctype + 'fcnt' + '.png')
            plt.close("all")
        
            p = document.add_paragraph()
            r = p.add_run()
            r.add_text(str(ctype) + '_fcnt')
            r.add_picture(ctype + 'fcnt' + '.png', width=Inches(6.0), height=Inches(2.))
            r.add_text('Equal no. of orthogonal Gr cells assigned to each class. From each group some spiked - weights updated for those Gr cell.')
            r.add_text('Y axis: Gr spike count, X axis: Class indices')
            r.add_text('Each figure depicts the number of Gr that spiked from each assigned groups for a particular gas class.')
            os.chdir(add)
            '''
            
            fig, axes = plt.subplots(1, nclasses, figsize=(18,6)) # no of classes
            for i in range(nclasses):
                for j in range(nclasses):
                    list_cnt[i][j] = np.asarray(list_cnt[i][j]).flatten()
                    axes[i].scatter(j* np.ones(len(list_cnt[i][j])) + 1, 
                                    list_cnt[i][j], color = color_list[i])
                    axes[j].set_ylim([0, len(param["idx_used_all"])])
                    axes[j].set_ylim([0, len(param["idx_used_all"])])
                    axes[j].set_xlabel(" Class indices.")
            plt.title(ctype + "listcnt")  
            
            os.chdir(data_add)
            fig.savefig(ctype + 'listcnt' + '.png')
            plt.close("all")
        
            p = document.add_paragraph()
            r = p.add_run()
            r.add_text(str(ctype) + '_listcnt')
            r.add_picture(ctype + 'listcnt' + '.png', width=Inches(6.0), height=Inches(2.))
            r.add_text('Same as above but here only Gr that were used for learning were included.')
            r.add_text('Equal no. of orthogonal Gr cells assigned to each class. From each group some spiked - weights updated for those Gr cell.')
            r.add_text('Y axis: Gr spike count, X axis: Class indices.')
            r.add_text('Each figure depicts the number of Gr that spiked from each assigned groups for a particular gas class.')
            os.chdir(add)
                               
    os.chdir(currdir)  

In [3]:
# spk_times()

In [4]:
def HORF():
    os.chdir(add)
    # import pdb;pdb.set_trace()
    syn_wght = pd.read_csv('syn_wght.csv', header=None)
    syn_wght = np.asarray(syn_wght.values)
    
    horf1 = []
    horf2 = []
    for i in range(np.size(syn_wght, 1)):
        idx = np.where(syn_wght[:, i] >= param["w_max"])[0]
        horf1.append(len(idx))
        idx = np.where(syn_wght[:, i] > 0.)[0]
        horf2.append(len(idx))

    horf1 = np.asarray(horf1).flatten()
    horf2 = np.asarray(horf2).flatten()
    
    fig, axes = plt.subplots(1, 2, figsize=(18,6))

    axes[0].hist(horf1)
    plt.title('Learnt HORF.')
    axes[0].set_ylim([0, 600])
    
    axes[1].hist(horf2)
    axes[1].set_ylim([0, 600])
    plt.title('HORF.')
    
    os.chdir(data_add)
    fig.savefig('HORF' + '.png')
    plt.close("all")

    p = document.add_paragraph()
    r = p.add_run()
    r.add_picture('HORF' + '.png', width=Inches(6.0), height=Inches(2.))
    r.add_text('(Left) Distribution of the no. of Mi connected to Gr with max. synaptic strength.')
    r.add_text('(Right) Distribution of the no. of Mi connected to Gr with non zero weights.')
    os.chdir(add)

In [5]:
def syn_dynamics():
    os.chdir(add)
    # import pdb;pdb.set_trace()
    fig = plt.figure()
    for k in range(min(3, len(param['idx_list_all'][0]))):
        w = np.load('t_resultsL0.npz')['syn_wght_time'][:, int(param['idx_list_all'][0][int(k)]), :]
        # color_w = iter(cm.jet(np.linspace(0, 1, np.size(w, 0))))
        for i in range(np.size(w, 0)):
        # c = next(color_w)
            # plt.plot(w[i, :], color=c)
            plt.plot(w[i, :])
            plt.ylim([-1., param["w_max"] + 1.])
        plt.ylabel('Mi to Gr Weight')
    
    os.chdir(data_add)
    fig.savefig('synw_time' + '.png')
    plt.close("all")

    p = document.add_paragraph()
    r = p.add_run()
    r.add_text('synw_time')
    r.add_picture('synw_time' + '.png', width=Inches(6.0), height=Inches(2.))
    r.add_text('Mi-Gr synaptic weight update profile - in a single gamma cycle.')
    os.chdir(add)

In [6]:
# HORF()

In [7]:
# syn_dynamics()

In [8]:
def predictions(classifier_type):
    
    for gp_idx, grp_typ in enumerate(group_type):
        fig, axes = plt.subplots(1, 2, figsize=(18,6))
        # idx_list = param["idx_list" + str(gp_idx)]
        idx_list = param["idx_list_all"]
        
        # Gas sensor data set
        os.chdir(add)
        num_gr = param["num_gr"]
        class_gr_bin = []
        y_ref  = y_fbL[gp_idx][0:(gp_idx+1)* param["nshots"]][:]
        n_trainexamples = len(y_ref)
        tmp_act_idx = []
        
        for i in range(n_trainexamples):
            tmp_list = np.zeros(num_gr)
            tmp_act_idx.extend(idx_list[i])
            tmp_list[idx_list[i]] = 1
            class_gr_bin.append(tmp_list[:].tolist())

        act_idx = np.unique(np.asarray(tmp_act_idx).flatten()).astype(int)
        os.chdir(currdir)

        AL_type = ['train_test', 'val_test', 'test_test']
        for idx, dtype in enumerate(AL_type):
            data_bin = []
            y_idT = [[] for _ in range(n_trainexamples)]
            y_idF = [[] for _ in range(n_trainexamples)]
            dist_T = [[] for _ in range(n_trainexamples)]
            y_pred = []
            dist_wr = []

            if idx == 0:
                y = y_fbL[gp_idx]
            elif idx == 1:
                y = y_val[gp_idx]
            elif idx == 2:
                y = y_test[gp_idx]
            os.chdir(add)
            for sample_idx in range(len(y)):
                count_gr = pd.read_csv('count_gr' + 'AL' + AL_type[idx] + 
                                       str(gp_idx) + str(sample_idx) + '.csv', header=None)
                count_gr_bin = np.asarray(count_gr.values.flatten())
                data_bin.append(count_gr_bin)
                os.chdir(add)

            ovlp = np.zeros((n_trainexamples, np.size(data_bin, 0)))
            for test_idx in range(np.size(data_bin, 0)):
                for i in range(n_trainexamples):
                    '''
                    ovlp[i, test_idx] = distance.hamming(
                        np.asarray(class_gr_bin[i])[param["gr_orth_idx"][y_fbL[gp_idx][i]]],
                        np.asarray(data_bin[test_idx])[param["gr_orth_idx"][y_fbL[gp_idx][i]]])
                    '''
                    if classifier_type == 'hamming':
                        '''
                        x1 = np.asarray(class_gr_bin[i])[param["idx_list_all"][y_ref[i]-1]]
                        x2 = np.asarray(data_bin[test_idx])[param["idx_list_all"][y_ref[i]-1]]
                        '''
                        '''
                        x1 = np.asarray(class_gr_bin[i])[param["idx_list_all"][i]]
                        x2 = np.asarray(data_bin[test_idx])[param["idx_list_all"][i]]
                        '''
                        x1 = np.asarray(class_gr_bin[i])
                        x2 = np.asarray(data_bin[test_idx])
                        ovlp[i, test_idx] = distance.hamming(x1, x2)
                    elif classifier_type == 'ovlp':
                        '''
                        x1 = np.asarray(class_gr_bin[i])[param["idx_list_all"][y_ref[i]-1]]
                        x2 = np.asarray(data_bin[test_idx])[param["idx_list_all"][y_ref[i]-1]]
                        '''
                        '''
                        x1 = np.asarray(class_gr_bin[i])[param["idx_list_all"][i]]
                        x2 = np.asarray(data_bin[test_idx])[param["idx_list_all"][i]]   
                        '''
                        x1 = np.asarray(class_gr_bin[i])
                        x2 = np.asarray(data_bin[test_idx])
                        if np.linalg.norm(x1) * np.linalg.norm(x2):
                            ovlp[i, test_idx] = (np.dot(x1, x2)/(np.linalg.norm(x1) * np.linalg.norm(x2)))
            
                if classifier_type == 'hamming':
                    if ovlp[np.argmin(ovlp[:, test_idx]), test_idx] > 0.5:
                        y_pred.append(0)
                    else:
                        y_pred.append(y_ref[np.argmin(ovlp[:, test_idx])])
                        
                        for target_idx, target_val in enumerate(y_ref):
                            if y_ref[np.argmin(ovlp[:, test_idx])] == target_val:
                                if y[test_idx] == target_val:
                                    y_idT[target_idx].append(test_idx)
                                    dist_T[target_idx].append(ovlp[np.argmin(ovlp[:, test_idx]), test_idx])
                                    all_rem_idx = np.setdiff1d(np.arange(len(y_ref)), 
                                                               np.argmin(ovlp[:, test_idx]))
                                    for index in all_rem_idx:
                                        dist_wr.append(ovlp[index, test_idx])
                                else:
                                    y_idF[target_idx].append(test_idx)
                        
                elif classifier_type == 'ovlp':
                    if ovlp[np.argmax(ovlp[:, test_idx]), test_idx] < 0.5:
                        y_pred.append(0)
                    else:
                        y_pred.append(y_ref[np.argmax(ovlp[:, test_idx])])
                       
                        for target_idx, target_val in enumerate(y_ref):
                            if y_ref[np.argmax(ovlp[:, test_idx])] == target_val:
                                if y[test_idx] == target_val:
                                    y_idT[target_idx].append(test_idx)
                                    dist_T[target_idx].append(ovlp[np.argmax(ovlp[:, test_idx]), test_idx])
                                    all_rem_idx = np.setdiff1d(np.arange(len(y_ref)), 
                                                               np.argmax(ovlp[:, test_idx]))
                                    for index in all_rem_idx:
                                        dist_wr.append(ovlp[index, test_idx])
                                else:
                                    y_idF[target_idx].append(test_idx) 
                     
           
            df = pd.DataFrame(y_pred)
            df.to_csv("pred" + classifier_type + str(idx) + "_" + str(gp_idx) + ".csv")
            os.chdir(data_add)
            df.to_csv("pred" + classifier_type + str(idx) + "_" + str(gp_idx) + ".csv")
            os.chdir(currdir)
            
            if classifier_type == 'ovlp' and gp_idx == (nclasses - 1):
                color_list = ["g", "r", "b", "k", "y", "c"]
                fig0, axes0 = plt.subplots(1, nclasses, figsize=(18,6))
                for i in range(np.size(ovlp, 1)):
                    for k in range(nclasses):
                        axes0[y[i]-1].plot(k, ovlp[k, i], marker='.', 
                                          linestyle='None', markersize=20, color = color_list[y[i]-1]) 
            
                os.chdir(data_add)
                fig0.savefig(dtype + classifier_type + '.png')
                plt.close("all")
                
                document.add_heading(dtype + '_overlap', level=1)
                p = document.add_paragraph()
                r = p.add_run()
                r.add_text(dtype + classifier_type)
                r.add_picture(dtype + classifier_type + '.png', width=Inches(6.0), height=Inches(2.))
                r.add_text(dtype + str(':') + 'X axis: Gas class indices, Y axis: value of overlap (normalized dot product).')
                r.add_text('Each figure describe the overlap value of Gr responses for a particular class of sensor input')
                os.chdir(add)
        '''  
        color_list = ["g", "r", "b", "k", "y", "c"]
        for i in range(n_trainexamples):
            axes[0].hist(np.asarray(dist_T[i]), color = color_list[i])
            axes[1].hist(np.asarray(dist_wr), color="m")
        axes[0].set_xlim([0., 1.])
        axes[0].set_ylim([0., 100.])
        axes[1].set_xlim([0., 1.])
        axes[1].set_ylim([0., 500.])
        # import pdb;pdb.set_trace()
        os.chdir(data_add)
        fig.savefig(grp_typ + classifier_type + 'hist' + '.png')
        plt.close("all")

        p = document.add_paragraph()
        r = p.add_run()
        r.add_text(grp_typ + dtype + classifier_type + 'hist')
        r.add_picture(grp_typ + classifier_type + 'hist' + '.png', width=Inches(6.0), height=Inches(2.))
        r.add_text('Group 1: Train class1, test on all, Group 2: Train on 1 & 2, test on all ...')
        r.add_text('(Left) Correct classification:' + classifier_type)
        r.add_text('(Right) Remaining incorrect values:' + classifier_type)
        os.chdir(add)
        '''
        
        

In [9]:
def comp_accuracy(classifier_type):
    
        test_type = ['fbL', 'val', 'test']
        for idx in range(len(test_type)):
            
            if test_type[idx] == 'test':
                comp_res = []
            
            document.add_heading('Results_' + classifier_type + '_' + test_type[idx], level=2)
            table = document.add_table(rows=10, cols=7)
            table.style = 'TableGrid'

            for i in range(nclasses):
                row = table.rows[i+1]
                row.cells[0].text = "class_" + str(i+1)
            row = table.rows[7]
            row.cells[0].text = 'None'
            row = table.rows[8]
            row.cells[0].text = 'Avg.'
            row = table.rows[9]
            row.cells[0].text = 'class Avg.'
            row = table.rows[0]
                
            for gp_idx, grp_typ in enumerate(group_type):
                # idx_list = param["idx_list" + str(gp_idx)]
                row.cells[gp_idx + 1].text = grp_typ

                os.chdir(add)
                if idx == 0:
                    df = y_fbL[gp_idx]
                elif idx == 1:
                    df = y_val[gp_idx]
                elif idx == 2:
                    df = y_test[gp_idx]

                for val in range(len(df)):
                    if df[val] > (gp_idx + 1):
                        df[val] = 0
                df_pred = pd.read_csv('pred' + classifier_type + str(idx) + "_" + 
                                      str(gp_idx) + '.csv', header=None)
                df_pred = df_pred[1][1:]
                os.chdir(currdir)

                pred = np.zeros(7)
                count = np.zeros(7)
                val = np.zeros(6)
                none_above  = 0
                none_all = 0

                for i in range(len(df)):
                    if df[i] == 0:
                        none_all = none_all + 1
                    else:
                        count[-1] = count[-1] + 1
                        count[df[i]-1] = count[df[i]-1] + 1

                    if df[i] == df_pred[i + 1]:
                        pred[-1] = pred[-1] + 1
                        if df[i] > 0:
                            pred[df[i]-1] = pred[df[i]-1] + 1
                        elif df[i] == 0:
                            none_above = none_above + 1
                    else:
                        if df[i] > 0:
                            val[df[i]-1] = val[df[i]-1] + 1
                
                print(test_type[idx], grp_typ)
                # import pdb;pdb.set_trace()
                for i in range(nclasses):
                    rowc = table.rows[i+1]
                    rowc.cells[gp_idx + 1].text = format((pred[i]/max(count[i], 1)) * 100, '.2f')
                    print((pred[i]/max(count[i], 1)) * 100)
                for i in range(nclasses):
                    print((val[i]/max(count[i], 1)) * 100)
                rowc = table.rows[7]
                rowc.cells[gp_idx + 1].text = format(((none_above/max(1, none_all)) * 100), '.2f')
                rowc = table.rows[8]
                rowc.cells[gp_idx + 1].text = format(((pred[-1]/(count[-1]+none_all)) * 100), '.2f')
                rowc = table.rows[9]
                class_pred = 0
                for gx in range(gp_idx+1):
                    class_pred = class_pred + pred[gx]
                rowc.cells[gp_idx + 1].text = format(((class_pred/(count[-1])) * 100), '.2f')
                if test_type[idx] == 'test':
                    # comp_res.append((class_pred/(count[-1])) * 100)
                    comp_res.append(format(((class_pred/(count[-1])) * 100), '.2f'))
                print(pred[-1], count[-1], none_above)
                print((pred[-1]/(count[-1]+none_all)) * 100)
                print((none_above/max(1, none_all)) * 100)
            if test_type[idx] == 'test':
                df = pd.DataFrame(comp_res)
                os.chdir(all_results)
                df.to_csv("comp_res" + "_" + str(batch_idx) + "_" + str(nshots) + "_" + str(ins) + "_" + str(classifier_type) + ".csv")
                os.chdir(currdir)
                

In [10]:

batch_idx = 1
ins=1
# shots = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
shots = [1]
for nshots in shots:
    currdir = os.getcwd()

    group_type = ['group1', 'group2', 'group3', 'group4', 'group5', 'group6']
    # stat = ['B4L', 'L', 'AL']
    stat = ['L', 'AL', 'AL']
    data_stat = ['train', 'val', 'test']
    AL = ['val_test', 'test_test']
    nclasses = 6
    path = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    path = os.path.normpath(path + os.sep + os.pardir)
    path = os.path.normpath(path + os.sep + os.pardir)
    all_results = os.path.join(path, 'NICE_results/drift/1')
    path = os.path.join(path, 'NICE_results/drift/1')
    
    os.chdir(path)
    idx_add = sorted(os.listdir())
    idx = [fname for fname in idx_add if fname.endswith('_run_parallel')][-3]
    data_idx = [fname for fname in idx_add if fname.endswith('__sim_info')][-3]
    
    os.chdir(currdir)
    add = "/".join([path, idx])
    data_add = "/".join([path, data_idx])
    os.chdir(add)
    with open('param.p', "rb") as handle:
        param = pickle.load(handle)
    os.chdir(currdir)
    data_path = os.path.normpath(os.getcwd() + os.sep + os.pardir)
    data_path = os.path.normpath(data_path + os.sep + os.pardir)
    data_path = os.path.normpath(data_path + os.sep + os.pardir)
    data_path = os.path.join(data_path, 'data')
    data_path = os.path.join(data_path, 'fneuro_data/' + str(nshots) + 
                             'shot/batch' + str(batch_idx) + '/set1') # input data location

    os.chdir(data_add)
    # document.add_heading('Ordinal forcing', level=1)
    document.add_heading('NTCE2', level=1)
    document.add_heading('Drift data - Batch1', level=1)
    document.add_heading('Important parameters', level=1)
    document.add_heading('ET+PG cells', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text("num_et=" + str(param["num_et"],) + '\t')
    r.add_text("num_et=" + str(param["num_pg"],) + '\t')
    document.add_heading('Apical Mitral cells', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text("num_apimi=" + str(param["num_apimi"],) + '\t')
    r.add_text("v_th_apimi=" + str(param["v_th_apimi"]) + '\t')
    document.add_heading('Mitral cells', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text("num_mi=" + str(param["num_mi"],) + '\t')
    r.add_text("mi_dup=" + str(param["mi_dup"],) + '\t')
    r.add_text("v_th_mi_min=" + str(param["v_th_mi_min"], ) + '\t')
    r.add_text("v_th_mi_max=" + str(param["v_th_mi_max"]) + '\t')
    r.add_text("v_th_gr_min=" + str(param["v_th_gr_min"], ) + '\t')
    r.add_text("v_th_gr_max=" + str(param["v_th_gr_max"]) + '\t')
    r.add_text("a_p_mg=" + str(param["a_p_mg"]) + '\t')
    r.add_text("tau_p_mg=" + str(param["tau_p_mg"]) + '\t')
    document.add_heading('Granule cells', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text("v_th_gr=" + str(param["v_th_gr"]) + '\t')
    # r.add_text("No of Gr assigned /class=" + str(len(param["gr_orth_idx"][1])) + '\t')
    r.add_text("num_gr=" + str(param["num_gr"]) + '\t')
    document.add_heading('Network', level=2)
    p = document.add_paragraph()
    r = p.add_run()
    r.add_text("Mi-Gr connection probability=" + str(param["cp"]) + '\t')
    r.add_text("Peak conductance change(gmax)=" + str(param["gmax_exc"]) + '\t')
    r.add_text("Initial synaptic weight=" + str(param["syn_wght"]) + '\t')
    # r.add_text("Max. allowed synaptic weight=" + str(param["w_max"]) + '\t')
    r.add_text("Oscillation frequency=" + str(param["f"]) + '\t')
    r.add_text("Oscillation offset parameter=" + str(param["offset_osc"]) + '\t')
    r.add_text("Oscillation sinusoid scaling parameter=" + str(param["amp_osc"]) + '\t')
    r.add_text("Nernst potential=" + str(param["en_exc"]) + '\t')
    r.add_text("Integration time step (dt)=" + str(param["dt"]) + '\t')
    os.chdir(add)

    y_train = [[] for _ in range(len(group_type))]
    y_fbL = [[] for _ in range(len(group_type))]
    y_val = [[] for _ in range(len(group_type))]
    y_test = [[] for _ in range(len(group_type))]
    idx_list = [[] for _ in range(len(group_type))]
    plot_typ = ['fbL', 'val', 'test']
    for idx, grp_typ in enumerate(group_type):
        # idx_list[idx] = param["idx_list_all" + str(idx)]
        idx_list[idx] = param["idx_list_all"]
        grp_path = os.path.join(data_path, grp_typ)
        os.chdir(grp_path)
        y_train[idx] = pd.read_csv('y_train.csv', header=None)
        y_train[idx] = y_train[idx][0]
        y_fbL[idx] = pd.read_csv('y_fbL.csv', header=None)
        y_fbL[idx] = y_fbL[idx][0]
        y_val[idx] = pd.read_csv('y_val.csv', header=None)
        y_val[idx] = y_val[idx][0]
        y_test[idx] = pd.read_csv('y_test.csv', header=None)
        y_test[idx] = y_test[idx][0]
        if idx == len(group_type)-1:
            document.add_heading('Input data', level=1)
            color_list = ["g", "r", "b", "k", "y", "c"]   
            fig, axes = plt.subplots(3, len(plot_typ), figsize=(18,6)) # no of classes
            for pidx, ptyp in enumerate(plot_typ):
                if pidx == 0:
                    y = y_fbL[idx]
                elif pidx == 1:
                    y = y_val[idx]
                elif pidx == 2:
                    y = y_test[idx]
                data = pd.read_csv(ptyp + '_data.csv', header=None)
                data = np.asarray(data.values)
                scaled_data = pd.read_csv(ptyp + '_data_scaled.csv', header=None)
                scaled_data = np.asarray(scaled_data.values)
                id_0 = np.flipud(np.argsort(data[0, :]))
                # print(pidx, np.shape(data), len(y))
                for i in range(len(y)):
                    if pidx == 0 and i == 0:
                        ref_data = data[i, :]
                        ref_sc_data = scaled_data[i, :]
                    axes[0, pidx].plot(np.transpose(data[i, :][id_0]), color = color_list[y[i]-1])
                    axes[1, pidx].plot(i, distance.euclidean(ref_data, data[i, :]),marker='.', 
                                       linestyle='None', markersize=20, color = color_list[y[i]-1])
                    axes[2, pidx].plot(i, distance.euclidean(ref_sc_data, scaled_data[i, :]),marker='.', 
                                       linestyle='None', markersize=20, color = color_list[y[i]-1])

                # axes[0, pidx].text(12, 350000, ptyp)
                axes[0, pidx].text(14, 600000, str(pidx))
                axes[1, pidx].text(2, 0.9, "raw")
                axes[2, pidx].text(2, 0.9, "scaled")
            os.chdir(data_add)
            fig.savefig('original_data' + '.png')
            plt.close("all")

            r = p.add_run()
            r.add_picture('original_data' + '.png', width=Inches(6.0), height=Inches(2.))
            r.add_text('Row 1(0,1,2): Raw sensor responses of train set, val set and test set.6 gas classes, 1 sample/class for train set.\n')
            r.add_text('Row 2 (0,1,2): Euclidean distances of raw sensor response samples w.r.t a reference sample  - sample 0 of train set.\n')
            r.add_text('Row 3(0, 1,2): Same as above but after sensor scaling - sensor scaling figures below.\n')
            os.chdir(grp_path)

            # document.add_heading('Input data', level=1)
            p = document.add_paragraph()
            plot_typ = ['fbL', 'val', 'test']
            for ptyp in plot_typ:
                r = p.add_run()
                r.add_picture('scaled_' + ptyp +'_dataprofile.png', width=Inches(6.0), height=Inches(2.))
                r.add_text(ptyp) 
                r.add_text('\nY axis: Sensor responses after sensor scaling, X axis: Sensor indices.')
                if ptyp == 'fbL':
                    p.add_run('Train set (after sensor scaling), 6 gas classes, 1 sample/class.')
                    p.add_run('Also used to generate sensor scaling parameters.')
                elif ptyp == 'val':
                     p.add_run('Validation set (after sensor scaling), Randomly selected 10 % of batch 1 data. ')
                elif ptyp == 'test':
                     p.add_run('Test set (after sensor scaling).')

        os.chdir(currdir)

    classifier_type = ['hamming', 'dot', 'ovlp', 'complexdot']
    predictions(classifier_type[0])
    comp_accuracy(classifier_type[0])
    predictions(classifier_type[2])
    comp_accuracy(classifier_type[2])
    os.chdir(data_add)
    document.save(str(data_idx) + 'results.docx')
    os.chdir(currdir)

/home/borthakur/anaconda3/envs/pytorch/lib/python3.7/site-packages/docx/styles/styles.py:143: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)


fbL group1
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0 1.0 0
16.666666666666664
0.0
fbL group2
100.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2.0 2.0 0
33.33333333333333
0.0
fbL group3
100.0
100.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.0 3.0 0
50.0
0.0
fbL group4
100.0
100.0
100.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.0 4.0 0
66.66666666666666
0.0
fbL group5
100.0
100.0
100.0
100.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
5.0 5.0 0
83.33333333333334
0.0
fbL group6
100.0
100.0
100.0
100.0
100.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
6.0 6.0 0
100.0
0.0
val group1
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
7.0 7.0 0
15.555555555555555
0.0
val group2
100.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
16.0 16.0 0
35.55555555555556
0.0
val group3
100.0
100.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
23.0 23.0 0
51.11111111111111
0.0
val group4
100.0
100.0
100.0
100.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
26.0 26.0 0
57.77777777777777
0.0
val group5
100.0
100.0
100.0
100.0
77.7777777